In [ ]:
import numpy as np
import os
import poly_yolo as yolo #or "import poly_yolo_lite as yolo" for the lite version
#import poly_yolo_lite as yolo

import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import keras
from keras.models import model_from_json, load_model

In [ ]:
#Set the learning phase to Test since the model is already trained.
keras.backend.set_learning_phase(0)

#load pretrained model
#if you want to detect more objects, lower the score and vice versa
trained_model = yolo.YOLO(model_path='models/poly_yolo.h5', iou=0.5, score=0.3)

In [ ]:
print(trained_model.yolo_model.layers[0].input)
print(trained_model.yolo_model.layers[-1].output)

In [ ]:
#save checkpoint
saver = tf.train.Saver()
tf_session = keras.backend.get_session()
input_graph_def = tf_session.graph.as_graph_def()
save_path = saver.save(tf_session, './checkpoint.ckpt')
tf.train.write_graph(input_graph_def,
                     './', 'poly_yolo_dpu.pb', as_text=False)

In [ ]:
#check the node names 
nodes_names = [node.name for node in 
               tf.get_default_graph().as_graph_def().node]
print(nodes_names)

In [ ]:
#!freeze_graph --help

In [ ]:
#Freeze Tensorflow graph: lastnode: concat_9; lastlayer: conv2d_69/BiasAdd
!freeze_graph \
    --input_meta_graph checkpoint.ckpt.meta \
    --input_checkpoint checkpoint.ckpt \
    --output_graph frozen.pb \
    --output_node_names conv2d_69/BiasAdd \
    --input_binary true

In [ ]:
#confirm available input and output node names
!vai_q_tensorflow inspect --input_frozen_graph=frozen.pb

#Prepare calibration data and function

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

dir_imgs_name = '../cityscapes/leftImg8bit_demoVideo/leftImg8bit/demoVideo/stuttgart_01/'  #path_where_are_images_to_clasification
list_of_imgs = [root+"/"+name for root, dirs, files in os.walk(dir_imgs_name) for name in files]    
list_of_imgs.sort()
num_imgs = 32

# get the 1st image with right shape
img = load_img(list_of_imgs[0])  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (1024, 2048, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 1024, 2048, 3)
dataset_calib = x
# fill in the following images
imgs          = 0
for im in range (1, num_imgs):#len(list_of_imgs)):
    imgs    += 1
    img = load_img(list_of_imgs[im])  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (1024, 2048, 3)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 1024, 2048, 3)
    dataset_calib = np.append(dataset_calib, x, axis=0)
print('Calibration data: {}, {}, {}'.format(dataset_calib.shape, dataset_calib.ndim, dataset_calib.dtype))

np.savez('./calib_data.npz', data = dataset_calib[:num_imgs])

del(dataset_calib)

In [ ]:
%%writefile input_func.py
import numpy as np

data = np.load('calib_data.npz')['data']

batch_size=1

def calib_input(iter):

    calib_data = data[iter*batch_size:(iter+1)*batch_size]

    return {'input_1': calib_data}

In [ ]:
#!vai_q_tensorflow quantize --help

In [ ]:
!vai_q_tensorflow quantize \
    --input_frozen_graph frozen.pb \
    --input_fn input_func.calib_input \
    --output_dir quantized \
    --input_nodes input_1 \
    --output_nodes conv2d_69/BiasAdd \
    --input_shapes ?,1024,2048,3 \
    --skip_check 0 \
    --simulate_dpu 1 \
    --calib_iter 1

In [ ]:
!vai_c_tensorflow \
    --frozen_pb quantized/deploy_model.pb \
    --arch ../../Ultra96.json \
    --output_dir . \
    --net_name poly_yolo_dpu_quantized